In [12]:
# Install all required Python packages.
# - pandas/numpy: data handling
# - scipy: the optimizer we'll use (L-BFGS-B)
# - requests: download data from the web/API
# - tqdm: nice progress bars (optional)

!pip install -q pandas numpy scipy requests tqdm

In [23]:
# =========================
# Imports (libraries we use)
# =========================
import io, math, sys
from dataclasses import dataclass
from typing import Dict, Tuple, List, Optional

import numpy as np               # numerical arrays
import pandas as pd              # tables/dataframes
import requests                  # HTTP calls to API
from scipy.optimize import minimize  # optimizer for MLE

# ================================================
# 1) Configuration and name normalization helpers
# ================================================

# We train on the last 3 EPL seasons from football-data.co.uk (E0 = Premier League)
SEASON_CODES = ["2223", "2324", "2425"]  # 22/23, 23/24, 24/25
FBD_URL = "https://www.football-data.co.uk/mmz4281/{season}/E0.csv"

# We fetch the 2025/26 Premier League fixtures (with dates) from football-data.org
FDORG_FIXTURES_URL = "https://api.football-data.org/v4/competitions/PL/matches?season=2025"

# Team name mapping so different sources “agree” (e.g., "Tottenham Hotspur FC" -> "Tottenham")
NAME_MAP = {
    "Manchester United": "Man United",
    "Manchester City": "Man City",
    "Tottenham Hotspur": "Tottenham",
    "Tottenham Hotspur FC": "Tottenham",
    "Brighton & Hove Albion": "Brighton",
    "Brighton & Hove Albion FC": "Brighton",
    "Brighton and Hove Albion": "Brighton",
    "Wolverhampton Wanderers": "Wolves",
    "Wolverhampton Wanderers FC": "Wolves",
    "Newcastle United": "Newcastle",
    "Newcastle United FC": "Newcastle",
    "West Ham United": "West Ham",
    "West Ham United FC": "West Ham",
    "Nottingham Forest": "Nott'm Forest",
    "Nottingham Forest FC": "Nott'm Forest",
    "AFC Bournemouth": "Bournemouth",
    "AFC Bournemouth FC": "Bournemouth",
    "Sheffield United": "Sheffield United",
    "Leeds United": "Leeds",
    "Leicester City": "Leicester",
}

# =========================================================
# 2) Data download utilities (results + fixtures with dates)
# =========================================================

def _download_csv(url: str) -> pd.DataFrame:
    """Download a CSV from the web and parse into a DataFrame."""
    r = requests.get(url, timeout=30, headers={"User-Agent": "Mozilla/5.0"})
    r.raise_for_status()                          # fail fast if 404/500
    return pd.read_csv(io.StringIO(r.text))       # parse CSV text into DataFrame

def load_last_3_seasons() -> pd.DataFrame:
    """Fetch and combine the last three EPL seasons for model training."""
    frames: List[pd.DataFrame] = []
    for code in SEASON_CODES:
        url = FBD_URL.format(season=code)         # build season URL
        print(f"Downloading {code} → {url}")
        df = _download_csv(url)                   # grab the CSV

        # keep only columns we need
        keep = [c for c in ["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG"] if c in df.columns]
        df = df[keep].copy()

        # parse dates (football-data uses day-first)
        if "Date" in df.columns:
            df["Date"] = pd.to_datetime(df["Date"], dayfirst=True, errors="coerce")

        # drop rows missing essentials
        df = df.dropna(subset=["HomeTeam", "AwayTeam", "FTHG", "FTAG"])

        # ensure goals are ints
        df["FTHG"] = df["FTHG"].astype(int)
        df["FTAG"] = df["FTAG"].astype(int)

        frames.append(df)

    return pd.concat(frames, ignore_index=True)   # combine seasons vertically

def get_fdorg_fixtures(api_key: str) -> pd.DataFrame:
    """Fetch official 2025/26 fixtures with dates from football-data.org using your API key."""
    if not api_key.strip():
        raise RuntimeError("API key missing. Set API_KEY in the previous cell.")
    # Call API with X-Auth-Token header (as per the email instructions)
    r = requests.get(
        FDORG_FIXTURES_URL,
        headers={"X-Auth-Token": api_key, "User-Agent": "Mozilla/5.0"},
        timeout=30
    )
    # Helpful messages on common issues
    if r.status_code == 401:
        raise RuntimeError("Unauthorized (401). Check your football-data.org API key.")
    if r.status_code != 200:
        raise RuntimeError(f"API error {r.status_code}: {r.text[:200]}")
    data = r.json()

    # Build a clean fixtures table: Date, HomeTeam, AwayTeam
    rows = []
    for m in data.get("matches", []):
        home = m.get("homeTeam", {}).get("name", "")
        away = m.get("awayTeam", {}).get("name", "")
        date = m.get("utcDate", None)
        if home and away:
            rows.append({"Date": date, "HomeTeam": home, "AwayTeam": away})

    df = pd.DataFrame(rows)
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")   # to pandas datetime
    df = df.dropna(subset=["HomeTeam", "AwayTeam"]).reset_index(drop=True)
    return df

# ==================================
# 3) Name harmonization (very common)
# ==================================

def harmonize_names(df: pd.DataFrame, cols=("HomeTeam", "AwayTeam")) -> pd.DataFrame:
    """Map provider-specific names to unified naming (football-data.co.uk style)."""
    def map_name(x: str) -> str:
        if pd.isna(x):
            return x
        x = str(x).strip()
        return NAME_MAP.get(x, x)

    out = df.copy()
    for c in cols:
        out[c] = out[c].map(map_name)
    return out

def fuzzy_map_to_seen(target: List[str], known: List[str]) -> Dict[str, str]:
    """If a team name isn't an exact match, map to the closest known team using fuzzy match."""
    from difflib import get_close_matches
    mapping: Dict[str, str] = {}
    for t in target:
        if t in known:
            mapping[t] = t
        else:
            cand = get_close_matches(t, known, n=1, cutoff=0.6)
            mapping[t] = cand[0] if cand else t
    return mapping

# ===========================================
# 4) Dixon–Coles model (Poisson goals + tweaks)
# ===========================================

@dataclass
class DCParams:
    """Container for model parameters."""
    attack: Dict[str, float]   # team attacking strengths
    defense: Dict[str, float]  # team defensive strengths
    home_adv: float            # league-wide home advantage
    rho: float                 # low-score dependency (Dixon–Coles correction)

def poisson_prob(lmbda: float, k: int) -> float:
    """Poisson probability mass function P(X=k)."""
    return math.exp(-lmbda) * (lmbda ** k) / math.factorial(k)

def tau_dc(x: int, y: int, lam_h: float, lam_a: float, rho: float) -> float:
    """Dixon–Coles low-score dependence adjustment factor."""
    if x == 0 and y == 0: return 1 - lam_h * lam_a * rho
    if x == 0 and y == 1: return 1 + lam_h * rho
    if x == 1 and y == 0: return 1 + lam_a * rho
    if x == 1 and y == 1: return 1 - rho
    return 1.0

def decay_weight(days_diff: float, xi: float = 0.0065) -> float:
    """Exponential time decay so recent matches matter more."""
    return math.exp(-xi * days_diff)

class DixonColesModel:
    """Fits attack/defense strengths + home advantage + rho via MLE with time decay."""
    def __init__(self, xi: float = 0.0065, max_goals: int = 6):
        self.xi = xi
        self.max_goals = max_goals
        self.params_: Optional[DCParams] = None
        self.teams_: List[str] = []

    def _pack(self, P: DCParams) -> np.ndarray:
        """Flatten parameters into a vector for the optimizer."""
        return np.array(
            [P.attack[t] for t in self.teams_] +
            [P.defense[t] for t in self.teams_] +
            [P.home_adv, P.rho],
            dtype=float
        )

    def _unpack(self, vec: np.ndarray) -> DCParams:
        """Rebuild structured params from the optimizer vector."""
        T = len(self.teams_)
        atk = dict(zip(self.teams_, vec[:T]))
        dfn = dict(zip(self.teams_, vec[T:2*T]))
        home_adv = float(vec[-2]); rho = float(vec[-1])

        # Center around zero for identifiability
        atk_mu = float(np.mean(list(atk.values())))
        dfn_mu = float(np.mean(list(dfn.values())))
        atk = {k: v - atk_mu for k, v in atk.items()}
        dfn = {k: v - dfn_mu for k, v in dfn.items()}
        return DCParams(atk, dfn, home_adv, rho)

    def _nll(self, vec: np.ndarray, df: pd.DataFrame) -> float:
        """Negative log-likelihood over all historical matches."""
        P = self._unpack(vec)
        ll = 0.0
        for _, row in df.iterrows():
            h, a = row["HomeTeam"], row["AwayTeam"]
            gh, ga = int(row["FTHG"]), int(row["FTAG"])
            days = float(row.get("_days", 0.0))
            w = decay_weight(days, self.xi)

            # Expected goals from team strengths
            lam_h = math.exp(P.home_adv + P.attack[h] - P.defense[a])
            lam_a = math.exp(P.attack[a] - P.defense[h])

            # Independent Poisson × DC adjustment (low-score correlation)
            base = poisson_prob(lam_h, gh) * poisson_prob(lam_a, ga)
            p = max(base * tau_dc(gh, ga, lam_h, lam_a, P.rho), 1e-12)

            ll += w * math.log(p)
        return -ll

    def fit(self, results: pd.DataFrame) -> "DixonColesModel":
        """Fit parameters by maximizing the time-decayed likelihood."""
        df = results.copy()
        self.teams_ = sorted(set(df["HomeTeam"]).union(set(df["AwayTeam"])))

        # Compute days-since for time decay
        if "Date" in df.columns and df["Date"].notna().any():
            max_date = df["Date"].max()
            df["_days"] = (max_date - df["Date"]).dt.days.astype(float)
        else:
            df["_days"] = 0.0

        # Reasonable starting guess
        init = DCParams(
            attack={t: 0.0 for t in self.teams_},
            defense={t: 0.0 for t in self.teams_},
            home_adv=0.2,
            rho=0.0,
        )
        x0 = self._pack(init)

        # Optimize with L-BFGS-B
        res = minimize(self._nll, x0, args=(df,), method="L-BFGS-B", options={"maxiter": 400})
        if not res.success:
            raise RuntimeError(f"Optimization failed: {res.message}")

        self.params_ = self._unpack(res.x)
        return self

    def team_rates(self, home: str, away: str) -> Tuple[float, float]:
        """Return expected goals (λ_home, λ_away) for a given pairing."""
        assert self.params_ is not None, "Model not fitted."
        P = self.params_
        lam_h = math.exp(P.home_adv + P.attack[home] - P.defense[away])
        lam_a = math.exp(P.attack[away] - P.defense[home])
        return lam_h, lam_a

    def score_matrix(self, home: str, away: str) -> np.ndarray:
        """Full matrix of scoreline probabilities up to max_goals (0..6 by default)."""
        lam_h, lam_a = self.team_rates(home, away)
        pmf_h = np.array([poisson_prob(lam_h, k) for k in range(self.max_goals + 1)])
        pmf_a = np.array([poisson_prob(lam_a, k) for k in range(self.max_goals + 1)])
        M = np.outer(pmf_h, pmf_a)

        # Apply low-score adjustments for (0,0), (0,1), (1,0), (1,1)
        for x in (0, 1):
            for y in (0, 1):
                M[x, y] *= tau_dc(x, y, lam_h, lam_a, self.params_.rho)  # type: ignore

        # Normalize (numerical safety)
        M = M / M.sum()
        return M

    def probs_hda(self, home: str, away: str) -> Tuple[float, float, float, Tuple[int, int]]:
        """Return P(Home), P(Draw), P(Away) and the most-likely scoreline."""
        M = self.score_matrix(home, away)
        p_home = float(np.sum([M[i, j] for i in range(M.shape[0]) for j in range(M.shape[1]) if i > j]))
        p_draw = float(np.sum([M[i, i] for i in range(M.shape[0])]))
        p_away = float(1.0 - p_home - p_draw)
        ix = np.unravel_index(np.argmax(M), M.shape)
        return p_home, p_draw, p_away, (int(ix[0]), int(ix[1]))

# ====================================================
# 5) Prediction helpers (map names, handle new teams)
# ====================================================

def league_avgs(P: DCParams) -> Tuple[float, float]:
    """Compute league-average attack/defense to use for unseen (promoted) teams."""
    return float(np.mean(list(P.attack.values()))), float(np.mean(list(P.defense.values())))

def predict_all(model: DixonColesModel, fixtures: pd.DataFrame) -> pd.DataFrame:
    """Predict probabilities and tips for every fixture row."""
    fx = harmonize_names(fixtures, cols=("HomeTeam", "AwayTeam"))  # unify names
    seen = set(model.teams_)                                      # teams seen in training
    P = model.params_; assert P is not None

    # fuzzy map to closest known names (handles small naming differences)
    mapping = fuzzy_map_to_seen(sorted(set(fx["HomeTeam"]) | set(fx["AwayTeam"])), list(seen))

    # add any unseen teams as league-average, so we still produce predictions
    atk_avg, def_avg = league_avgs(P)
    for t in set(fx["HomeTeam"]).union(set(fx["AwayTeam"])):
        mapped = mapping.get(t, t)
        if mapped not in seen:
            P.attack[mapped] = atk_avg
            P.defense[mapped] = def_avg
            model.teams_.append(mapped)
            seen.add(mapped)

    # compute probabilities for each fixture
    rows = []
    for _, r in fx.iterrows():
        h = mapping.get(r["HomeTeam"], r["HomeTeam"])
        a = mapping.get(r["AwayTeam"], r["AwayTeam"])
        p_h, p_d, p_a, (sh, sa) = model.probs_hda(h, a)
        tip = "H" if p_h >= max(p_d, p_a) else ("D" if p_d >= max(p_h, p_a) else "A")
        rows.append({
            "Date": r.get("Date", pd.NaT),
            "HomeTeam": h,
            "AwayTeam": a,
            "P(Home)": round(p_h, 4),
            "P(Draw)": round(p_d, 4),
            "P(Away)": round(p_a, 4),
            "MostLikelyScore": f"{sh}-{sa}",
            "Tip": tip
        })
    return pd.DataFrame(rows).sort_values(["Date", "HomeTeam", "AwayTeam"]).reset_index(drop=True)

# ===========================================
# 6) Full pipeline: train → fetch fixtures → predict
# ===========================================

def run_pipeline_direct(api_key: str) -> pd.DataFrame:
    """Train on last 3 seasons, fetch 2025/26 fixtures with dates, and predict all matches."""
    print("Step 1/3: Download last 3 seasons (football-data.co.uk)")
    results = load_last_3_seasons()

    print("Step 2/3: Fit Dixon–Coles model (this can take ~10–30s)")
    model = DixonColesModel(xi=0.0065, max_goals=6).fit(results)

    print("Step 3/3: Fetch official 2025/26 fixtures (football-data.org) and predict")
    fixtures = get_fdorg_fixtures(api_key)

    preds = predict_all(model, fixtures)
    return preds

In [24]:
# This runs the end-to-end pipeline using the API_KEY from Step 2.
preds = run_pipeline_direct(API_KEY)

# Peek at the first few rows to confirm the format:
preds.head(10)

Step 1/3: Download last 3 seasons (football-data.co.uk)
Step 2/3: Fit Dixon–Coles model (this can take ~10–30s)
Step 3/3: Fetch official 2025/26 fixtures (football-data.org) and predict


,Date,HomeTeam,AwayTeam,P(Home),P(Draw),P(Away),MostLikelyScore,Tip
0,2025-08-15 19:00:00+00:00,Liverpool,Bournemouth,0.5589,0.2160,0.2251,1-1,H
1,2025-08-16 11:30:00+00:00,Aston Villa,Newcastle,0.3573,0.2442,0.3985,1-1,A
2,2025-08-16 14:00:00+00:00,Brighton,Fulham,0.4822,0.2194,0.2984,1-1,H
3,2025-08-16 14:00:00+00:00,Sunderland AFC,West Ham,0.3559,0.2751,0.3691,1-1,A
4,2025-08-16 14:00:00+00:00,Tottenham,Burnley,0.5230,0.2193,0.2577,1-1,H
5,2025-08-16 16:30:00+00:00,Wolves,Man City,0.1811,0.2099,0.6089,0-1,A
6,2025-08-17 13:00:00+00:00,Chelsea,Crystal Palace,0.4702,0.2571,0.2727,1-1,H
7,2025-08-17 13:00:00+00:00,Nott'm Forest,Brentford,0.3864,0.2416,0.3720,1-1,H
8,2025-08-17 15:30:00+00:00,Man United,Arsenal,0.1757,0.2380,0.5863,0-1,A
9,2025-08-18 19:00:00+00:00,Sheffield United,Everton,0.1138,0.1971,0.6890,0-1,A


In [25]:
# Save the predictions to a CSV file.
preds.to_csv("predictions_epl_2025_26.csv", index=False)
"Saved to predictions_epl_2025_26.csv"

'Saved to predictions_epl_2025_26.csv'

In [27]:
import pandas as pd
df = pd.read_csv("predictions_epl_2025_26.csv")
len(df), df["HomeTeam"].nunique(), df["AwayTeam"].nunique()

(380, 20, 20)